<a href="https://colab.research.google.com/github/anujavenkatachalam04/anujavenkatachalam04/blob/main/notebooks/extract_remote_sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract remote sensing data - NDVI, NDWI, NDBI from sentinel 2

In [ ]:
# # import dependencies
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [1]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import uuid
import ee
from concurrent.futures import ThreadPoolExecutor, as_completed


Mounted at /content/drive


In [7]:
os.chdir("/content/drive/MyDrive/CHVI")

In [4]:
# --- Initialize Earth Engine ---
# ee.Authenticate()
# ee.Initialize()

In [8]:
# --- Paths ---
grid_path = "5_Shapefiles/Rajasthan_10kmx10km_grid.geojson"
output_folder = "1_Data/Remote_Sensing/Raw"
os.makedirs(output_folder, exist_ok=True)

In [9]:
# Import 10kmx10km grid
grid = gpd.read_file(grid_path)

In [12]:
grid.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
grid["grid_id"] = grid.index + 1  # unique ID

In [14]:
# --- Define monthly range ---
start_date = "2024-01-01"
end_date   = "2025-09-30"
date_range = pd.date_range(start=start_date, end=end_date, freq="M")

/tmp/ipython-input-3492290752.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq="M")


In [15]:
date_range

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31',
               '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31',
               '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30',
               '2025-05-31', '2025-06-30', '2025-07-31', '2025-08-31',
               '2025-09-30'],
              dtype='datetime64[ns]', freq='ME')

# NDVI - B8, B4

In [17]:
# --- NDVI helper for one feature ---
def get_ndvi_for_feature(row, ee_start_date):
    geom = ee.Geometry.Polygon(row.geometry.__geo_interface__["coordinates"])
    s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
          .filterDate(ee_start_date, ee_start_date.advance(1, "month"))
          .filterBounds(geom)
          .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
          .map(lambda img: img.normalizedDifference(["B8", "B4"]).rename("NDVI")))
    ndvi_img = s2.mean()
    try:
        ndvi_val = ndvi_img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=10
        ).getInfo().get("NDVI", None)
    except Exception:
        ndvi_val = None
    return {
        "grid_id": row["grid_id"],
        "NDVI": ndvi_val
    }

# --- Process one month ---
def process_month(current_date):
    ee_start_date = ee.Date(current_date.strftime("%Y-%m-%d"))
    results = []

    # Parallel execution (I/O bound: Earth Engine calls)
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(get_ndvi_for_feature, row, ee_start_date): idx
            for idx, row in grid.iterrows()
        }
        for f in tqdm(as_completed(futures), total=len(futures),
                      desc=f"Processing {current_date.strftime('%Y-%m')}"):
            results.append(f.result())

    # Collect results
    df = pd.DataFrame(results)
    df["Date"] = current_date
    output_path = f"{output_folder}/ndvi_{current_date.strftime('%Y-%m')}.csv"
    df.to_csv(output_path, index=False)
    print(f"Saved NDVI data for {current_date.strftime('%Y-%m')} → {output_path}")

# --- Run monthly loop ---
for dt in date_range:
    process_month(dt)

print("All NDVI files generated successfully.")


# NDWI - B3, B8

In [ ]:
# --- NDWI helper for one feature ---
def get_ndwi_for_feature(row, ee_start_date):
    geom = ee.Geometry.Polygon(row.geometry.__geo_interface__["coordinates"])
    s2 = (
        ee.ImageCollection("COPERNICUS/S2_SR")
        .filterDate(ee_start_date, ee_start_date.advance(1, "month"))
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .map(lambda img: img.normalizedDifference(["B3", "B8"]).rename("NDWI"))
    )
    ndwi_img = s2.mean()
    try:
        ndwi_val = (
            ndwi_img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geom,
                scale=10
            ).getInfo().get("NDWI", None)
        )
    except Exception:
        ndwi_val = None
    return {"grid_id": row["grid_id"], "NDWI": ndwi_val}

# --- Process one month ---
def process_month_ndwi(current_date):
    ee_start_date = ee.Date(current_date.strftime("%Y-%m-%d"))
    results = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(get_ndwi_for_feature, row, ee_start_date): idx
            for idx, row in grid.iterrows()
        }
        for f in tqdm(as_completed(futures), total=len(futures),
                      desc=f"NDWI {current_date.strftime('%Y-%m')}"):
            results.append(f.result())

    df = pd.DataFrame(results)
    df["Date"] = current_date
    output_path = f"{output_folder}/ndwi_{current_date.strftime('%Y-%m')}.csv"
    df.to_csv(output_path, index=False)
    print(f"Saved NDWI data for {current_date.strftime('%Y-%m')} → {output_path}")

# --- Run monthly loop ---
for dt in date_range:
    process_month_ndwi(dt)

print("All NDWI files generated successfully.")


# NDBI - B11, B8

In [ ]:
# --- NDBI helper for one feature ---
def get_ndbi_for_feature(row, ee_start_date):
    geom = ee.Geometry.Polygon(row.geometry.__geo_interface__["coordinates"])
    s2 = (
        ee.ImageCollection("COPERNICUS/S2_SR")
        .filterDate(ee_start_date, ee_start_date.advance(1, "month"))
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .map(lambda img: img.normalizedDifference(["B11", "B8"]).rename("NDBI"))
    )
    ndbi_img = s2.mean()
    try:
        ndbi_val = (
            ndbi_img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geom,
                scale=10
            ).getInfo().get("NDBI", None)
        )
    except Exception:
        ndbi_val = None
    return {"grid_id": row["grid_id"], "NDBI": ndbi_val}

# --- Process one month ---
def process_month_ndbi(current_date):
    ee_start_date = ee.Date(current_date.strftime("%Y-%m-%d"))
    results = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(get_ndbi_for_feature, row, ee_start_date): idx
            for idx, row in grid.iterrows()
        }
        for f in tqdm(as_completed(futures), total=len(futures),
                      desc=f"NDBI {current_date.strftime('%Y-%m')}"):
            results.append(f.result())

    df = pd.DataFrame(results)
    df["Date"] = current_date
    output_path = f"{output_folder}/ndbi_{current_date.strftime('%Y-%m')}.csv"
    df.to_csv(output_path, index=False)
    print(f"Saved NDBI data for {current_date.strftime('%Y-%m')} → {output_path}")

# --- Run monthly loop ---
for dt in date_range:
    process_month_ndbi(dt)

print("All NDBI files generated successfully.")
